In [110]:
import warnings
import itertools
import pandas as pd
import seaborn as sns
import geopandas as gpd
import matplotlib.pyplot as plt
warnings.filterwarnings("ignore")

## **Tasas crudas de mortalidad**

Se calcularon las tasas crudas de mortalidad por cada 100.000 personas, para cada departamento y por año, utilizando la siguiente fórmula:

$$
T_{kj} = \left( \frac{n_{kj}}{p_j} \right) \times k
$$

donde:

- $ T_{kj} $ es la tasa cruda de mortalidad para el año j y el departamento correspondiente.  
- $ n_{kj}$ es el número de muertes registradas.  
- $ p_j $ es la población total en ese año.  
- $ k $ es el factor de estandarización, en este caso $ k = 100.000$.


```{note}
Es importante tener en cuenta que el conjunto de datos utilizado solo registra muertes en mujeres mayores de 40 años, en el periodo comprendido entre los años 2009 y 2023. Por esta razón, las poblaciones utilizadas para el cálculo de las tasas corresponden exclusivamente a mujeres de 40 años o más, dentro de ese intervalo temporal.
```

In [111]:
url_muertes = 'https://github.com/sePerezAlbor/Data/blob/main/MUERTES-CAN-MAMA-2009-2023.xlsx?raw=true'
url_poblacion = 'https://github.com/sePerezAlbor/Data/blob/main/POB-M40-DEP-2009-2023.xlsx?raw=true'  

muertes = pd.read_excel(url_muertes)
poblacion = pd.read_excel(url_poblacion)

In [112]:
poblacion['DPNOM'] = poblacion['DPNOM'].str.upper()

In [113]:
equivalencias = {
    'SANTAFE DE BOGOTA D.C': 'BOGOTA DC',
    'BOGOTÁ, D.C.': 'BOGOTA DC',
    'ARCHIPIELAGO DE SAN ANDRES PROVIDENCIA Y SANTA CATALINA': 'ARCHIPIÉLAGO DE SAN ANDRÉS',
    'BOLIVAR': 'BOLÍVAR',
    'BOYACA': 'BOYACÁ',
    'ATLANTICO': 'ATLÁNTICO',
    'CORDOBA': 'CÓRDOBA',
    'GUAINIA': 'GUAINÍA',
    'GUAVIARE': 'GUAVIARE',
    'VAUPES': 'VAUPÉS',
    'VICHADA': 'VICHADA',
    'NORTE DE SANTANDER': 'NORTE DE SANTANDER',
    'CHOCÓ': 'CHOCÓ'
}
muertes['DPNOM'] = muertes['DPNOM'].replace(equivalencias)
poblacion['DPNOM'] = poblacion['DPNOM'].replace(equivalencias)

In [114]:
muertes.head()

,DPNOM,ANIO,FALLECIDAS
0,AMAZONAS,2009,0
1,AMAZONAS,2010,0
2,AMAZONAS,2011,1
3,AMAZONAS,2012,0
4,AMAZONAS,2013,1


In [115]:
# Crea una nueva columna con el total de mujeres mayores de 40
cols_mujeres = [col for col in poblacion.columns if col.startswith('Mujeres_')]
poblacion['TOTAL_MUJERES'] = poblacion[cols_mujeres].replace(',', '', regex=True).astype(float).sum(axis=1)

In [116]:
poblacion.head()

,DP,DPNOM,ANIO,Mujeres_40,Mujeres_41,Mujeres_42,Mujeres_43,Mujeres_44,Mujeres_45,Mujeres_46,...,Mujeres_77,Mujeres_78,Mujeres_79,Mujeres_80,Mujeres_81,Mujeres_82,Mujeres_83,Mujeres_84,Mujeres_85 y más,TOTAL_MUJERES
0,5,ANTIOQUIA,2009,42534,43316,43935,44189,43894,43221,42324,...,7752,7247,6762,6304,5837,5342,4875,4391,24849,1055525.0
1,5,ANTIOQUIA,2010,41697,42555,43320,43923,44161,43850,43166,...,7925,7402,6892,6401,5937,5469,4979,4519,26098,1085742.0
2,5,ANTIOQUIA,2011,40942,41718,42560,43309,43896,44117,43794,...,8097,7568,7032,6519,6022,5556,5089,4608,27312,1114816.0
3,5,ANTIOQUIA,2012,40363,40957,41717,42543,43276,43848,44058,...,8274,7735,7189,6641,6126,5627,5162,4701,28444,1142775.0
4,5,ANTIOQUIA,2013,39973,40383,40962,41704,42514,43230,43792,...,8557,7909,7351,6790,6234,5720,5224,4760,29503,1169906.0


In [117]:
departamentos_amazona = ['AMAZONAS', 'GUAINÍA', 'GUAVIARE', 'VAUPÉS', 'VICHADA']

amazona_df = poblacion[poblacion['DPNOM'].isin(departamentos_amazona)]
amazona_muertes = muertes[muertes['DPNOM'].isin(departamentos_amazona)]

suma_por_año = (amazona_df.groupby('ANIO', as_index=False).sum(numeric_only=True))
suma_por_año_muertes = (amazona_muertes.groupby('ANIO', as_index=False).sum(numeric_only=True))

suma_por_año['DPNOM'] = 'GRUPO AMAZONA'
suma_por_año_muertes['DPNOM'] = 'GRUPO AMAZONA'

cols = ['DPNOM', 'ANIO'] + [col for col in suma_por_año.columns if col not in ['DPNOM', 'ANIO']]
suma_por_año = suma_por_año[cols]

cols_muertes = ['DPNOM', 'ANIO'] + [col for col in suma_por_año_muertes.columns if col not in ['DPNOM', 'ANIO']]
suma_por_año_muertes = suma_por_año_muertes[cols_muertes]

poblacion_con_grupo = pd.concat([poblacion, suma_por_año], ignore_index=True)
poblacion_con_grupo_muertes = pd.concat([muertes, suma_por_año_muertes], ignore_index=True)

poblacion_solo_con_grupo = poblacion_con_grupo[~poblacion_con_grupo['DPNOM'].isin(departamentos_amazona)]
poblacion_solo_con_grupo_muertes = poblacion_con_grupo_muertes[~poblacion_con_grupo_muertes['DPNOM'].isin(departamentos_amazona)]

muertes = poblacion_solo_con_grupo_muertes.copy()
poblacion = poblacion_solo_con_grupo.copy()

In [118]:
def calcular_tasa_mortalidad(muertes, poblacion, año_inicio=2009, año_fin=2023, k=100000):
    """
    Calcula la tasa cruda de mortalidad por departamento para mujeres mayores de 40 años.

    Parámetros:
    - muertes: DataFrame con columnas ['DPNOM', 'ANIO', 'FALLECIDAS']
    - poblacion: DataFrame con columnas ['DPNOM', 'ANIO', 'TOTAL_MUJERES']
    - año_inicio: Año inicial del periodo (int)
    - año_fin: Año final del periodo (int)
    - k: Factor de estandarización (int, default 100000)

    Retorna:
    - DataFrame con columnas ['DPNOM', 'FALLECIDAS', 'TOTAL_MUJERES', 'Tasa_Mortalidad']
    """
    # Filtrado de años
    muertes_filtrado = muertes[(muertes['ANIO'] >= año_inicio) & (muertes['ANIO'] <= año_fin)]
    pob_filtrada = poblacion[(poblacion['ANIO'] >= año_inicio) & (poblacion['ANIO'] <= año_fin)]

    # Agrupación de muertes
    muertes_agg = muertes_filtrado.groupby('DPNOM', as_index=False)['FALLECIDAS'].sum()

    # # Agrupación de población
    if año_inicio == año_fin:
        pob_agg = pob_filtrada.groupby('DPNOM', as_index=False)['TOTAL_MUJERES'].sum()
    else:
        pob_agg = pob_filtrada.groupby('DPNOM', as_index=False)['TOTAL_MUJERES'].median()

    # Unión y cálculo de la tasa
    tasa = pd.merge(muertes_agg, pob_agg, on='DPNOM')
    tasa['Tasa_Mortalidad'] = (tasa['FALLECIDAS'] / tasa['TOTAL_MUJERES']) * k

    return tasa


In [119]:
tasa_mortalidad  = calcular_tasa_mortalidad(muertes, poblacion, año_inicio=2009, año_fin=2023, k = 100000)

In [120]:
tasa_mortalidad

,DPNOM,FALLECIDAS,TOTAL_MUJERES,Tasa_Mortalidad
0,ANTIOQUIA,6879,1252145.0,549.377269
1,ARAUCA,147,34028.0,431.997179
2,ARCHIPIÉLAGO DE SAN ANDRÉS,36,12514.0,287.677801
3,ATLÁNTICO,3483,441159.0,789.511265
4,BOGOTA DC,9169,1474342.0,621.904551
5,BOLÍVAR,1758,336762.0,522.030395
6,BOYACÁ,820,247768.0,330.954764
7,CALDAS,1120,224800.0,498.220641
8,CASANARE,175,56366.0,310.470851
9,CAUCA,705,244525.0,288.314078


In [121]:
# Resultados: mayor, menor y promedio
mayor = tasa_mortalidad.loc[tasa_mortalidad['Tasa_Mortalidad'].idxmax()]
menor = tasa_mortalidad.loc[tasa_mortalidad['Tasa_Mortalidad'].idxmin()]
promedio = tasa_mortalidad['Tasa_Mortalidad'].mean()

print("\nMayor tasa:", mayor['DPNOM'], f"{mayor['Tasa_Mortalidad']:.2f}")
print("Menor tasa:", menor['DPNOM'], f"{menor['Tasa_Mortalidad']:.2f}")
print("Promedio:", f"{promedio:.2f}")


Mayor tasa: ATLÁNTICO 789.51
Menor tasa: GRUPO AMAZONA 121.66
Promedio: 450.82


## **Tasas ajustadas por edad**

Para estimar tasas ajustadas de mortalidad por cáncer y poder realizar comparaciones válidas entre diferentes regiones o periodos de tiempo, se utilizó el método directo de ajuste por edad, tomando como referencia la **población estándar de Segi (1960)**. Esta población fue diseñada para facilitar la comparación internacional de tasas de mortalidad y morbilidad, y se basa en una distribución teórica de población que representa una media ponderada de estructuras poblacionales de distintas regiones del mundo en ese momento.  
El ajuste por edad corrige las posibles distorsiones generadas por diferencias en la estructura etaria de las poblaciones observadas, haciendo más precisa la comparación del riesgo de morir por una causa específica, como el cáncer, entre distintos departamentos o periodos.  
La tasa ajustada se expresa por cada **100.000 habitantes**, lo cual es un estándar internacional que facilita la comunicación y análisis de datos epidemiológicos.

Ahmad, O. B., Boschi-Pinto, C., Lopez, A. D., Murray, C. J., Lozano, R., & Inoue, M. (2001). Age standardization of rates: a new WHO standard. Geneva: World Health Organization. GPE Discussion Paper Series: No. 31.


| Grupo de Edad        | Intervalo                   | Proporción Segi (%) |
|----------------------|-----------------------------|----------------------|
| **Grupo_edad1**      | 40–49 años                  | 6.0 + 6.0 = **12.00** |
| **Grupo_edad2**      | 50–59 años                  | 5.0 + 4.0 = **9.00**  |
| **Grupo_edad3**      | 60–69 años                  | 4.0 + 3.0 = **7.00**  |
| **Grupo_edad4**      | 70 años en adelante (70+)   | 2.0 + 1.0 + 0.5 + 0.5 = **4.00** |



In [122]:
url_muertes_complete = 'https://github.com/sePerezAlbor/Data/blob/main/data_to_tasas.csv?raw=true'
muertes_complete = pd.read_csv(url_muertes_complete)
muertes_complete.drop(columns=['Unnamed: 0'], inplace=True) # Eliminar la columna innecesaria
muertes_complete.head()

,region_Def,Nombre_Departamento_Def,Nombre_Municipio_Def,Nombre_Departamento_Res,Nombre_Municipio_Res,sitio_def,año_def,mes_def,sexo,estado_civil,grupo_edad,nivel_edu,seg_social,asistencia_med,causa_basica,Fecha
0,Andina,ANTIOQUIA,MEDELLÍN,ANTIOQUIA,MEDELLÍN,Casa,2009,1,Femenino,"Unión Libre, Divorciado/Otro",40-54 años,Secundaria,Subsidiado,SI,Parte no especificada,2009-01
1,Andina,ANTIOQUIA,MEDELLÍN,ANTIOQUIA,MEDELLÍN,Casa,2009,1,Femenino,Viudo,65-74 años,Primaria,Contributivo,SI,Parte no especificada,2009-01
2,Andina,ANTIOQUIA,MEDELLÍN,ANTIOQUIA,SABANETA,Hospital o Clínica,2009,1,Femenino,Casado,40-54 años,Secundaria,Contributivo,SI,Parte no especificada,2009-01
3,Andina,ANTIOQUIA,MEDELLÍN,ANTIOQUIA,BELLO,Hospital o Clínica,2009,1,Femenino,Casado,40-54 años,Secundaria,Contributivo,SI,Parte no especificada,2009-01
4,Andina,ANTIOQUIA,MEDELLÍN,ANTIOQUIA,MEDELLÍN,Casa,2009,1,Femenino,"Unión Libre, Divorciado/Otro",40-54 años,Secundaria,Contributivo,SI,Parte no especificada,2009-01


In [123]:
columnas_interes = ['Nombre_Departamento_Def', 'año_def', 'grupo_edad']
muertes_simple = muertes_complete[columnas_interes]
conteos = (muertes_simple.groupby(['Nombre_Departamento_Def', 'grupo_edad']).size().reset_index(name='count'))
muertes_complete = conteos.copy()
muertes_complete.head()

,Nombre_Departamento_Def,grupo_edad,count
0,AMAZONAS,40-54 años,7
1,AMAZONAS,55-64 años,1
2,AMAZONAS,65-74 años,1
3,AMAZONAS,75+ años,2
4,ANTIOQUIA,40-54 años,1284


In [124]:
departamentos_amazona = ['AMAZONAS', 'GUAINÍA', 'GUAVIARE', 'VAUPÉS', 'VICHADA']
amazona_df = muertes_complete[muertes_complete['Nombre_Departamento_Def'].isin(departamentos_amazona)]
amazona_df = (amazona_df.groupby([ 'grupo_edad']).sum(numeric_only=True).reset_index())
amazona_df['Nombre_Departamento_Def'] = 'GRUPO AMAZONA'
orden_columnas = ['Nombre_Departamento_Def', 'grupo_edad', 'count']
amazona_df = amazona_df[orden_columnas]
amazona_df

,Nombre_Departamento_Def,grupo_edad,count
0,GRUPO AMAZONA,40-54 años,21
1,GRUPO AMAZONA,55-64 años,7
2,GRUPO AMAZONA,65-74 años,4
3,GRUPO AMAZONA,75+ años,7


In [125]:
muertes_complete = muertes_complete[~muertes_complete['Nombre_Departamento_Def'].isin(departamentos_amazona)].copy()
muertes_complete = pd.concat([muertes_complete, amazona_df], ignore_index=True)

In [126]:
departamentos = muertes_complete['Nombre_Departamento_Def'].unique()
grupos_edad = muertes_complete['grupo_edad'].unique()
combinaciones = pd.DataFrame(list(itertools.product(departamentos, grupos_edad)), columns=['Nombre_Departamento_Def','grupo_edad'])
resultado_completo = combinaciones.merge(muertes_complete,on=['Nombre_Departamento_Def',  'grupo_edad'],how='left')
resultado_completo['count'] = resultado_completo['count'].fillna(0).astype(int)
resultado_completo.head()

,Nombre_Departamento_Def,grupo_edad,count
0,ANTIOQUIA,40-54 años,1284
1,ANTIOQUIA,55-64 años,1620
2,ANTIOQUIA,65-74 años,1571
3,ANTIOQUIA,75+ años,1711
4,ARAUCA,40-54 años,46


In [127]:
resultado_completo.rename(columns={'Nombre_Departamento_Def': 'DPNOM'}, inplace=True)
resultado_completo.replace({'DPNOM': {'ARCHIPIÉLAGO DE SAN ANDRÉS Y PROVIDENCIA Y SANTA CATALINA': 'ARCHIPIÉLAGO DE SAN ANDRÉS'}}, inplace=True)

In [128]:
poblacion[cols_mujeres] = poblacion[cols_mujeres].replace(',', '', regex=True).astype(float)

poblacion['Grupo_edad1'] = poblacion[[f'Mujeres_{edad}' for edad in range(40, 55)]].sum(axis=1)
poblacion['Grupo_edad2'] = poblacion[[f'Mujeres_{edad}' for edad in range(55, 65)]].sum(axis=1)
poblacion['Grupo_edad3'] = poblacion[[f'Mujeres_{edad}' for edad in range(65, 75)]].sum(axis=1)

cols_edad4 = [f'Mujeres_{edad}' for edad in range(75, 85)] + ['Mujeres_85 y más']
poblacion['Grupo_edad4'] = poblacion[cols_edad4].sum(axis=1)


In [129]:
columnas_interes = ['DPNOM','Grupo_edad1', 'Grupo_edad2', 'Grupo_edad3', 'Grupo_edad4', 'TOTAL_MUJERES']
poblacion_interes = poblacion[columnas_interes].copy()

In [130]:
# Paso 2: Reformatear la base de población (melt)
poblacion_melted = poblacion_interes.melt(id_vars=['DPNOM'],value_vars=['Grupo_edad1', 'Grupo_edad2', 'Grupo_edad3', 'Grupo_edad4'],
                                          var_name='grupo_edad',value_name='poblacion')

# Paso 3: Mapear nombres de grupo de edad (para que coincidan con los de resultado_completo)
grupo_mapping = {'Grupo_edad1': '40-54 años','Grupo_edad2': '55-64 años','Grupo_edad3': '65-74 años','Grupo_edad4': '75+ años'}
poblacion_melted['grupo_edad'] = poblacion_melted['grupo_edad'].map(grupo_mapping)
poblacion_melted = poblacion_melted.groupby(['DPNOM', 'grupo_edad'], as_index=False).median(numeric_only=True)
poblacion_melted

,DPNOM,grupo_edad,poblacion
0,ANTIOQUIA,40-54 años,625833.0
1,ANTIOQUIA,55-64 años,327265.0
2,ANTIOQUIA,65-74 años,186605.0
3,ANTIOQUIA,75+ años,112442.0
4,ARAUCA,40-54 años,19801.0
...,...,...,...
111,TOLIMA,75+ años,30921.0
112,VALLE DEL CAUCA,40-54 años,448714.0
113,VALLE DEL CAUCA,55-64 años,236331.0
114,VALLE DEL CAUCA,65-74 años,142252.0


In [131]:
poblacion_melted[poblacion_melted['DPNOM'] == 'ANTIOQUIA']
data_resumen = pd.merge(resultado_completo, poblacion_melted, on=['DPNOM', 'grupo_edad'], how='left')
data_resumen.head()

,DPNOM,grupo_edad,count,poblacion
0,ANTIOQUIA,40-54 años,1284,625833.0
1,ANTIOQUIA,55-64 años,1620,327265.0
2,ANTIOQUIA,65-74 años,1571,186605.0
3,ANTIOQUIA,75+ años,1711,112442.0
4,ARAUCA,40-54 años,46,19801.0


In [132]:
datos_tasa_edad = data_resumen.copy()

In [133]:
proporcion_segi = {
    '40-54 años': 12,
    '55-64 años': 9,
    '65-74 años': 7,
    '75+ años': 4
}

# Normalizamos la proporción Segi
total_p = sum(proporcion_segi.values())
proporcion_segi_normalizada = {k: v / total_p for k, v in proporcion_segi.items()}
proporcion_segi_normalizada

{'40-54 años': 0.375,
 '55-64 años': 0.28125,
 '65-74 años': 0.21875,
 '75+ años': 0.125}

In [134]:
# Añadimos la proporción Segi y calculamos la tasa específica por grupo etario
datos_tasa_edad['proporcion_segi'] = datos_tasa_edad['grupo_edad'].map(proporcion_segi_normalizada)
datos_tasa_edad['tasa_cruda'] = datos_tasa_edad['count'] / datos_tasa_edad['poblacion']
datos_tasa_edad['tasa_ajustada_parcial'] = datos_tasa_edad['proporcion_segi'] * datos_tasa_edad['tasa_cruda']

# Agrupamos por departamento y multiplicamos por 100000
tae_por_departamento = datos_tasa_edad.groupby('DPNOM')['tasa_ajustada_parcial'].sum() 
tae_por_departamento = tae_por_departamento * 100000
tae_por_departamento = tae_por_departamento.reset_index(name='TAE')
tae_por_departamento

,DPNOM,TAE
0,ANTIOQUIA,590.531071
1,ARAUCA,494.872446
2,ARCHIPIÉLAGO DE SAN ANDRÉS,420.434978
3,ATLÁNTICO,836.909500
4,BOGOTA DC,679.403134
5,BOLÍVAR,547.514060
6,BOYACÁ,329.271414
7,CALDAS,502.813331
8,CAQUETÁ,439.002864
9,CASANARE,366.925311


In [135]:
tasas = pd.merge(tasa_mortalidad, tae_por_departamento, on='DPNOM', how='left')
tasas

,DPNOM,FALLECIDAS,TOTAL_MUJERES,Tasa_Mortalidad,TAE
0,ANTIOQUIA,6879,1252145.0,549.377269,590.531071
1,ARAUCA,147,34028.0,431.997179,494.872446
2,ARCHIPIÉLAGO DE SAN ANDRÉS,36,12514.0,287.677801,420.434978
3,ATLÁNTICO,3483,441159.0,789.511265,836.909500
4,BOGOTA DC,9169,1474342.0,621.904551,679.403134
5,BOLÍVAR,1758,336762.0,522.030395,547.514060
6,BOYACÁ,820,247768.0,330.954764,329.271414
7,CALDAS,1120,224800.0,498.220641,502.813331
8,CASANARE,175,56366.0,310.470851,366.925311
9,CAUCA,705,244525.0,288.314078,289.526946


## **SMR: Standard Mortality Rate**

$$
\text{SMR} = \frac{\text{Número observado de muertes}}{\text{Número esperado de muertes}}
$$

### **SMR Por departamento**

**NOTA:** Más adelante se agregarán covariables para el cálculo del SMR.

Moraga, P. (s.f.). Indirect standardization to calculate expected cases. Recuperado el 24 de abril de 2025, de https://www.paulamoraga.com/presentation-course/#/indirect-standardiz.-to-calculate-expected-cases

In [136]:
tasas.head()

,DPNOM,FALLECIDAS,TOTAL_MUJERES,Tasa_Mortalidad,TAE
0,ANTIOQUIA,6879,1252145.0,549.377269,590.531071
1,ARAUCA,147,34028.0,431.997179,494.872446
2,ARCHIPIÉLAGO DE SAN ANDRÉS,36,12514.0,287.677801,420.434978
3,ATLÁNTICO,3483,441159.0,789.511265,836.909500
4,BOGOTA DC,9169,1474342.0,621.904551,679.403134


In [137]:
muertes_totales_nal = tasas['FALLECIDAS'].median()
mujeres_totales_nal = tasas['TOTAL_MUJERES'].median()
tasa_nacional = muertes_totales_nal / mujeres_totales_nal

In [138]:
tasas['MUERTES_ESPERADAS'] = tasas['TOTAL_MUJERES'] * tasa_nacional
tasas['SMR'] = tasas['FALLECIDAS'] / tasas['MUERTES_ESPERADAS']*100

In [139]:
tasas

,DPNOM,FALLECIDAS,TOTAL_MUJERES,Tasa_Mortalidad,TAE,MUERTES_ESPERADAS,SMR
0,ANTIOQUIA,6879,1252145.0,549.377269,590.531071,5486.489435,125.380721
1,ARAUCA,147,34028.0,431.997179,494.872446,149.099555,98.591843
2,ARCHIPIÉLAGO DE SAN ANDRÉS,36,12514.0,287.677801,420.434978,54.832251,65.654792
3,ATLÁNTICO,3483,441159.0,789.511265,836.909500,1933.014302,180.184906
4,BOGOTA DC,9169,1474342.0,621.904551,679.403134,6460.083941,141.933140
5,BOLÍVAR,1758,336762.0,522.030395,547.514060,1475.580827,119.139526
6,BOYACÁ,820,247768.0,330.954764,329.271414,1085.638256,75.531605
7,CALDAS,1120,224800.0,498.220641,502.813331,985.000000,113.705584
8,CASANARE,175,56366.0,310.470851,366.925311,246.977358,70.856698
9,CAUCA,705,244525.0,288.314078,289.526946,1071.428492,65.800005


### **Interpretación del SMR**

- **SMR = 100**: Mortalidad observada igual a la esperada.
- **SMR > 100**: Más muertes de las esperadas (mayor riesgo).
- **SMR < 100**: Menos muertes de las esperadas (menor riesgo).


## **Suavizado de tasas**

## **ÍNDICE DE MORAN**

## **Modelos geográficamente ponderados**